# <font color='blue'>Data Science Academy - Matemática Para Machine Learning</font>

## Estudo de Caso 5 - Compreendendo Como Funciona o Backpropagation

## Parte 4 - Otimização com Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) é uma versão de Gradient Descent, onde em cada passagem para a frente, obtemos um lote de dados com amostras aleatórias do conjunto de dados total. Aqui onde entra em cena o batch_size. Esse é o tamanho do lote. Idealmente, todo o conjunto de dados seria alimentado na rede neural em cada passagem para a frente, mas na prática isso acaba não sendo possível, devido a restrições de memória. SGD é uma aproximação de Gradient Descent, quanto mais lotes processados pela rede neural, melhor será a aproximação.

Uma implementação do SGD envolve:

1. Gerar lotes de dados de amostras aleatórias do conjunto de dados total.

2. Executar a rede para frente (Forward Pass) e para trás (Backward pass) para calcular o gradiente (com dados de (1)).

3. Aplicar a atualização de descida do gradiente.

4. Repetir as etapas 1-3 até a convergência ou o loop for parado por outro mecanismo (como o número de épocas, por exemplo).

Se tudo correr bem, a perda da rede vai diminuindo, indicando pesos e bias mais úteis ao longo do tempo.

In [1]:
import numpy as np

class Neuronio:
    """
    Classe base para os nós da rede.

    Argumentos:

        "nodes_entrada": Uma lista de nós com arestas para este nó.
    """
    def __init__(self, nodes_entrada = []):
        """
        O construtor do nó (é executado quando o objeto é instanciado). 
        Define propriedades que podem ser usadas por todos os nós.
        """
        # Lista de nós com arestas para este nó.
        self.nodes_entrada = nodes_entrada
        
        # Lista de nós para os quais este nó gera saída.
        self.nodes_saida = []
        
        # O valor calculado por este nó. É definido executando o método forward().
        self.valor = None
        
        # Este objeto é um dicionário com pares chaves/valor entre {} 
        # As chaves (keys) são os inputs para este nó e o valores (values) são as paciais deste nó em relação ao input.
        self.gradientes = {}
        
        # Configuramos este nó como um nó de saída para todos os nós de entrada.
        for n in nodes_entrada:
            n.nodes_saida.append(self)

    def forward(self):
        """
        Todo o nó que usar essa classe como uma classe base, precisa definir seu próprio método "forward".
        """
        raise NotImplementedError

    def backward(self):
        """
        Todo o nó que usar essa classe como uma classe base, precisa definir seu próprio método "backward".
        """
        raise NotImplementedError



class Input(Neuronio):
    """
    Input genérico para a rede.
    """
    def __init__(self):
        # O construtor da classe base deve ser executado para configurar todas as propriedades aqui.
        #
        # A propriedade mais importante de Input é valor.
        # self.valor é definido na função topological_sort().
        Neuronio.__init__(self)

    def forward(self):
        # Nada a ser feito aqui.
        pass

    def backward(self):
        # Um nó de Input não possui entradas (pois ele já é a entrada) e assim o gradiente (derivada) é zero.
        # A palavra reservada "self", é referência para este objeto.
        self.gradientes = {self: 0}
        
        # Pesos e bias podem ser inputs, assim precisamos somar o gradiente de outros gradientes de saída
        for n in self.nodes_saida:
            self.gradientes[self] += n.gradientes[self]
            

class Linear(Neuronio):
    """
    Representa um nó que realiza transformação linear.
    """
    def __init__(self, X, W, b):
        # O construtor da classe base (nó). 
        # Pesos e bias são tratados como nós de entrada (nodes_entrada).
        Neuronio.__init__(self, [X, W, b])

    def forward(self):
        """
        Executa a matemática por trás da transformação linear.
        """
        X = self.nodes_entrada[0].valor
        W = self.nodes_entrada[1].valor
        b = self.nodes_entrada[2].valor
        self.valor = np.dot(X, W) + b

    def backward(self):
        """
        Calcula o gradiente com base nos valores de saída.
        """
        # Inicializa um parcial para cada um dos nodes_entrada.
        self.gradientes = {n: np.zeros_like(n.valor) for n in self.nodes_entrada}
        
        # Ciclo através dos outputs. 
        # O gradiente mudará dependendo de cada output, assim os gradientes são somados sobre todos os outputs.
        for n in self.nodes_saida:
            
            # Obtendo parcial da perda em relação a este nó.
            grad_cost = n.gradientes[self]
            
            # Definindo o parcial da perda em relação às entradas deste nó.
            self.gradientes[self.nodes_entrada[0]] += np.dot(grad_cost, self.nodes_entrada[1].valor.T)
            
            # Definindo o parcial da perda em relação aos pesos deste nó.
            self.gradientes[self.nodes_entrada[1]] += np.dot(self.nodes_entrada[0].valor.T, grad_cost)
            
            # Definindo o parcial da perda em relação ao bias deste nó.
            self.gradientes[self.nodes_entrada[2]] += np.sum(grad_cost, axis = 0, keepdims = False)


class Sigmoid(Neuronio):
    """
    Representa o nó da função de ativação Sigmoid.
    """
    def __init__(self, node):
        # O construtor da classe base.
        Neuronio.__init__(self, [node])

    def _sigmoid(self, x):
        """
        Este método é separado do `forward` porque ele também será usado com "backward".

        `x`: Um array Numpy.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Executa a função _sigmoid e define a variável self.valor
        """
        input_value = self.nodes_entrada[0].valor
        self.valor = self._sigmoid(input_value)

    def backward(self):
        """
        Calcula o gradiente usando a derivada da função sigmoid 
        
        O método backward da classe Sigmoid, soma as derivadas (é uma derivada normal quando há apenas uma variável) 
        em relação à única entrada sobre todos os nós de saída.
        """
        
        # Inicializa os gradientes com zero.
        self.gradientes = {n: np.zeros_like(n.valor) for n in self.nodes_entrada}
        
        # Soma a parcial em relação ao input sobre todos os outputs.
        for n in self.nodes_saida:
            grad_cost = n.gradientes[self]
            sigmoid = self.valor
            self.gradientes[self.nodes_entrada[0]] += sigmoid * (1 - sigmoid) * grad_cost


class MSE(Neuronio):
    def __init__(self, y, a):
        """
        Função de custo para calcular o erro médio quadrático.
        Deve ser usado como último nó da rede.
        """
        # Chamada ao construtor da classe base.
        Neuronio.__init__(self, [y, a])

    def forward(self):
        """
        Calcula o erro médio ao quadrado.
        """
        # Fazemos o reshape para evitar possíveis problemas nas operações de matrizes/vetores 
        #
        # Convertendo os 2 arrays (3,1) garantimos que o resultado será (3,1) e, assim, 
        # teremos uma subtração elementwise.
        y = self.nodes_entrada[0].valor.reshape(-1, 1)
        a = self.nodes_entrada[1].valor.reshape(-1, 1)

        self.m = self.nodes_entrada[0].valor.shape[0]
        
        # Salva o output computado para o backward pass.
        self.diff = y - a
        self.valor = np.mean(self.diff**2)

    def backward(self):
        """
        Calcula o gradiente do custo.
        """
        self.gradientes[self.nodes_entrada[0]] = (2 / self.m) * self.diff
        self.gradientes[self.nodes_entrada[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Classifica os nós em ordem topológica usando o Algoritmo de Kahn.

    `Feed_dict`: um dicionário em que a chave é um nó `Input` e o valor é o respectivo feed de valor para esse nó.

    Retorna uma lista de nós ordenados.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.nodes_saida:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.valor = feed_dict[n]

        L.append(n)
        for m in n.nodes_saida:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Executa uma passagem para a frente e uma passagem para trás através de uma lista de nós ordenados.

     Argumentos:

         `Graph`: O resultado de `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # O valor negativo no slice permite fazer uma cópia da mesma lista na ordem inversa.
    for n in graph[::-1]:
        n.backward()


def sgd_update(params, learning_rate = 1e-2):
    """
    Atualiza o valor de cada parâmetro treinável com o SGD.

    Argumentos:

         `Trainables`: uma lista de nós `Input` que representam pesos / bias.
         `Learning_rate`: a taxa de aprendizado.
    """
    # Executa o SGD
    #
    # Loop sobre todos os parâmetros
    for t in params:
        # Alterar o valor do parâmetro, subtraindo a taxa de aprendizado 
        # multiplicado pela parte do custo em relação a esse parâmetro
        partial = t.gradientes[t]
        t.valor -= learning_rate * partial

### Executando o Grafo

http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

In [2]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

# Carrega os dados
data = load_boston()

# Variáveis de entrada e saída para treinamento supervisionado
X_ = data['data']
y_ = data['target']

# Normaliza os dados
X_ = (X_ - np.mean(X_, axis = 0)) / np.std(X_, axis = 0)

# Número de features e número de neurônios
n_features = X_.shape[1]
n_hidden = 10

# Define valores randômicos para inicializar pesos e bias
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Rede Neural
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

# Define o feed_dict
feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

# Número de epochs (altere esse valor para ver as mudanças no resultado)
epochs = 1000

# Número total de exemplos
m = X_.shape[0]

# Batch size
batch_size = 11
steps_per_epoch = m // batch_size

# Define o grafo computacional
graph = topological_sort(feed_dict)

# Valores que serão aprendidos pela rede
params = [W1, b1, W2, b2]

# Número total de exemplos
print("Número Total de Exemplos = {}".format(m))

# Treinamento do modelo
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        
        # Passo 1 - Testa aleatoriamente um lote de exemplos
        X_batch, y_batch = resample(X_, y_, n_samples = batch_size)

        # Reset dos valores de X e y 
        X.valor = X_batch
        y.valor = y_batch

        # Passo 2 - Forward e Backpropagation
        forward_and_backward(graph)

        # Passo 3 - Otimização por SGD
        sgd_update(params)

        loss += graph[-1].valor

    print("Epoch: {}, Custo: {:.3f}".format(i+1, loss/steps_per_epoch))

Número Total de Exemplos = 506
Epoch: 1, Custo: 137.029
Epoch: 2, Custo: 36.451
Epoch: 3, Custo: 21.682
Epoch: 4, Custo: 23.894
Epoch: 5, Custo: 23.786
Epoch: 6, Custo: 20.405
Epoch: 7, Custo: 26.579
Epoch: 8, Custo: 18.325
Epoch: 9, Custo: 15.669
Epoch: 10, Custo: 20.981
Epoch: 11, Custo: 17.762
Epoch: 12, Custo: 15.604
Epoch: 13, Custo: 13.883
Epoch: 14, Custo: 15.487
Epoch: 15, Custo: 11.780
Epoch: 16, Custo: 13.371
Epoch: 17, Custo: 13.429
Epoch: 18, Custo: 13.689
Epoch: 19, Custo: 12.558
Epoch: 20, Custo: 11.345
Epoch: 21, Custo: 12.382
Epoch: 22, Custo: 13.404
Epoch: 23, Custo: 12.184
Epoch: 24, Custo: 8.530
Epoch: 25, Custo: 10.271
Epoch: 26, Custo: 9.166
Epoch: 27, Custo: 9.317
Epoch: 28, Custo: 9.497
Epoch: 29, Custo: 10.519
Epoch: 30, Custo: 9.466
Epoch: 31, Custo: 10.189
Epoch: 32, Custo: 10.157
Epoch: 33, Custo: 7.484
Epoch: 34, Custo: 6.629
Epoch: 35, Custo: 9.137
Epoch: 36, Custo: 9.142
Epoch: 37, Custo: 9.709
Epoch: 38, Custo: 8.723
Epoch: 39, Custo: 7.594
Epoch: 40, Cus

Epoch: 331, Custo: 4.225
Epoch: 332, Custo: 5.827
Epoch: 333, Custo: 4.569
Epoch: 334, Custo: 5.172
Epoch: 335, Custo: 6.251
Epoch: 336, Custo: 5.488
Epoch: 337, Custo: 5.164
Epoch: 338, Custo: 5.357
Epoch: 339, Custo: 5.216
Epoch: 340, Custo: 4.264
Epoch: 341, Custo: 5.154
Epoch: 342, Custo: 5.440
Epoch: 343, Custo: 4.976
Epoch: 344, Custo: 5.280
Epoch: 345, Custo: 3.859
Epoch: 346, Custo: 5.237
Epoch: 347, Custo: 5.763
Epoch: 348, Custo: 4.513
Epoch: 349, Custo: 4.867
Epoch: 350, Custo: 5.420
Epoch: 351, Custo: 4.977
Epoch: 352, Custo: 5.673
Epoch: 353, Custo: 5.480
Epoch: 354, Custo: 4.537
Epoch: 355, Custo: 5.448
Epoch: 356, Custo: 5.496
Epoch: 357, Custo: 5.395
Epoch: 358, Custo: 5.943
Epoch: 359, Custo: 5.282
Epoch: 360, Custo: 4.822
Epoch: 361, Custo: 5.285
Epoch: 362, Custo: 4.584
Epoch: 363, Custo: 5.669
Epoch: 364, Custo: 4.446
Epoch: 365, Custo: 5.778
Epoch: 366, Custo: 5.136
Epoch: 367, Custo: 4.858
Epoch: 368, Custo: 5.439
Epoch: 369, Custo: 5.530
Epoch: 370, Custo: 4.942


Epoch: 663, Custo: 4.705
Epoch: 664, Custo: 4.652
Epoch: 665, Custo: 4.913
Epoch: 666, Custo: 4.194
Epoch: 667, Custo: 4.680
Epoch: 668, Custo: 3.743
Epoch: 669, Custo: 4.703
Epoch: 670, Custo: 4.300
Epoch: 671, Custo: 4.308
Epoch: 672, Custo: 4.876
Epoch: 673, Custo: 4.324
Epoch: 674, Custo: 4.177
Epoch: 675, Custo: 4.740
Epoch: 676, Custo: 4.642
Epoch: 677, Custo: 3.824
Epoch: 678, Custo: 4.589
Epoch: 679, Custo: 4.239
Epoch: 680, Custo: 5.144
Epoch: 681, Custo: 4.002
Epoch: 682, Custo: 4.991
Epoch: 683, Custo: 4.733
Epoch: 684, Custo: 4.294
Epoch: 685, Custo: 4.018
Epoch: 686, Custo: 4.803
Epoch: 687, Custo: 4.877
Epoch: 688, Custo: 4.864
Epoch: 689, Custo: 4.324
Epoch: 690, Custo: 4.085
Epoch: 691, Custo: 4.417
Epoch: 692, Custo: 4.999
Epoch: 693, Custo: 4.253
Epoch: 694, Custo: 4.066
Epoch: 695, Custo: 4.552
Epoch: 696, Custo: 4.311
Epoch: 697, Custo: 4.949
Epoch: 698, Custo: 4.276
Epoch: 699, Custo: 5.195
Epoch: 700, Custo: 4.381
Epoch: 701, Custo: 4.919
Epoch: 702, Custo: 4.547


## Fim